In [11]:
import esm
from esm.sdk.api import ESMProtein, GenerationConfig

You need an account on: https://forge.evolutionaryscale.ai/

After you have made an account, log in, click on "API Keys" and create a new key.

In [ ]:
# Enter your API key here. But keep in mind this is a security risk.
# Never add your key to a github repo or post it publicly in any way.
api_key = "Your API key here"

In [16]:
model = esm.sdk.client("esm3-medium-2024-08", token=api_key)

In [17]:
# Define a prompt with masked tokens (underscores) for the part you want to generate
prompt_sequence = "____________________________________DQATSLRILNNGHAFNVEFDDSQDKAVLKGGPLDGTYRLIQFHFHWGSLDGQGSEHTVDKKKYAAELHLVHWNTKYGDFGKAVQQPDGLAVLGIFLKVGSAKPGLQKVVDVLDSIKTKGKSADFTNFDPRGLLPESLDYWTYPGSLTTPP___________________________________________________________"
protein = ESMProtein(sequence=prompt_sequence)

# Generate the protein sequence
protein = model.generate(
    protein,
    GenerationConfig(
        track="sequence",  # specify that we want to generate a sequence
        num_steps=8,      # number of steps for iterative decoding
        temperature=0.7   # controls diversity vs. perplexity
    )
)

# The `protein` object now contains the generated sequence.
# You can access it via `protein.sequence`
print(protein.sequence)


MKDYPACSGKRQSPIDIVTSKVTKVSLPPLEFTGYDDQATSLRILNNGHAFNVEFDDSQDKAVLKGGPLDGTYRLIQFHFHWGSLDGQGSEHTVDKKKYAAELHLVHWNTKYGDFGKAVQQPDGLAVLGIFLKVGSAKPGLQKVVDVLDSIKTKGKSADFTNFDPRGLLPESLDYWTYPGSLTTPPCSESVTWIVFKEPIEISKEQLQKFRTSLFFEEEGKDEKLLVNNFRPVQPLNGRTVKESS


To fold the protein you have created, go to: https://alphafoldserver.com/

You need to set up an account there as well.